In [ ]:
### Data
import xarray as xr
import cfgrib
import numpy as np
from wrf import smooth2d

### Plotting
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import matplotlib.ticker as mticker
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
import matplotlib.transforms as mtransforms

### Warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def get_slp(filepath, min_lat=-90, max_lat=90, min_lon=-180, max_lon=180):
    
    file = xr.open_dataset(filepath, engine='cfgrib',
                backend_kwargs={'filter_by_keys': {'typeOfLevel': 'meanSea'}})
    var = file.get('prmsl').to_dataframe()
    latitudes = var.index.get_level_values('latitude')
    longitudes = var.index.get_level_values('longitude')
    map_function = lambda lon: (lon - 360) if (lon > 180) else lon
    remapped_longitudes = longitudes.map(map_function)
    var['longitude'] = remapped_longitudes
    var['latitude'] = latitudes
    lat_filter = (var['latitude'] >= min_lat) & (var['latitude'] <= max_lat)
    lon_filter = (var['longitude'] >= min_lon) & (var['longitude'] <= max_lon)
    var = var.loc[lat_filter & lon_filter]
    var = var.set_index(['latitude', 'longitude']).to_xarray()
    
    return var

def get_z(filepath, min_lat=-90, max_lat=90, min_lon=-180, max_lon=180):
    
    file = xr.open_dataset(filepath, engine='cfgrib',
                backend_kwargs={'filter_by_keys': {'typeOfLevel': 'isobaricInhPa'}})
    var = file.get('gh')[4].to_dataframe()
    latitudes = var.index.get_level_values('latitude')
    longitudes = var.index.get_level_values('longitude')
    map_function = lambda lon: (lon - 360) if (lon > 180) else lon
    remapped_longitudes = longitudes.map(map_function)
    var['longitude'] = remapped_longitudes
    var['latitude'] = latitudes
    lat_filter = (var['latitude'] >= min_lat) & (var['latitude'] <= max_lat)
    lon_filter = (var['longitude'] >= min_lon) & (var['longitude'] <= max_lon)
    var = var.loc[lat_filter & lon_filter]
    var = var.set_index(['latitude', 'longitude']).to_xarray()
    
    return var

cols = [
    'red',
    'blue',
    'green',
    'orange',
    'purple',
    'brown',
    'pink',
    'gray',
    'olive',
    'cyan',
    'magenta',
    'lime',
    'teal',
    'indigo',
    'gold',
    'orchid',
    'crimson',
    'navy',
    'turquoise',
    'coral'
]

In [ ]:
### Number of ensemble members
nens = 20

### Z500
min_lat1 = 28
max_lat1 = 44
min_lon1 = -97
max_lon1 = -78
z_dict = {}
for i in range(1,nens+1):
    if i < 10:
        filename = '/p/work1/lloveras/nov2018/gefs_data/fcs_2018111312/gep0'+str(i)+'.t12z.pgrb2af48'
    else:
        filename = '/p/work1/lloveras/nov2018/gefs_data/fcs_2018111312/gep'+str(i)+'.t12z.pgrb2af48'

    z_dict[i-1] = get_z(filename, min_lat1, max_lat1, min_lon1, max_lon1)

z_anl = get_z('/p/work1/lloveras/nov2018/gefs_data/anl_2018111512/gec00.t12z.pgrb2aanl',
                  min_lat1, max_lat1, min_lon1, max_lon1)

### SLP
min_lat2 = 32
max_lat2 = 48
min_lon2 = -82
max_lon2 = -63
slp_dict = {}
for i in range(1,nens+1):
    if i < 10:
        filename = '/p/work1/lloveras/nov2018/gefs_data/fcs_2018111312/gep0'+str(i)+'.t12z.pgrb2af72'
    else:
        filename = '/p/work1/lloveras/nov2018/gefs_data/fcs_2018111312/gep'+str(i)+'.t12z.pgrb2af72'

    slp_dict[i-1] = get_slp(filename, min_lat2, max_lat2, min_lon2, max_lon2)

slp_anl = get_slp('/p/work1/lloveras/nov2018/gefs_data/anl_2018111612/gec00.t12z.pgrb2aanl',
                  min_lat2, max_lat2, min_lon2, max_lon2)

In [ ]:
fig, axd = plt.subplot_mosaic([['left','right']],
                              constrained_layout=True, figsize=(7.0,3.6), dpi=200, 
                              gridspec_kw={'width_ratios':[1,1],'height_ratios':[1]},
                              per_subplot_kw={'left':{'projection':crs.PlateCarree()},
                                              'right':{'projection':crs.PlateCarree()}})

trans = mtransforms.ScaledTranslation(1/30, -1/30, fig.dpi_scale_trans)

##############
### LEFT
#############

axd['left'].set_extent((min_lon1,max_lon1,min_lat1,max_lat1))

for i in range(nens):
    axd['left'].contour(z_dict[i]['longitude'], z_dict[i]['latitude'], z_dict[i]['gh']/10,
                  levels=[550],
                  colors=cols[i],transform=crs.PlateCarree())

cs1 = axd['left'].contour(z_anl['longitude'], z_anl['latitude'], z_anl['gh']/10,
                  levels=np.arange(0,1500,10),
                  colors='k',transform=crs.PlateCarree())
axd['left'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,10),fontsize=6,colors='k',zorder=-1)

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['left'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['left'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Add the gridlines
gls = axd['left'].gridlines(draw_labels=True, dms=True,
                   x_inline=False, y_inline=False,linestyle='dashed')
gls.top_labels=False
gls.bottom_labels=True
gls.right_labels=False
gls.left_labels=True
gls.xlocator = mticker.FixedLocator([-95, -90, -85, -80])
gls.ylocator = mticker.FixedLocator([26, 31, 36, 41])

axd['left'].set_title('1200 UTC 15 Nov 2018 ($t$ = 48 h)')
axd['left'].text(0.0, 1.0, '(a) Z500',transform=axd['left'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### RIGHT
#############

axd['right'].set_extent((min_lon2,max_lon2,min_lat2,max_lat2))

for i in range(nens):
    axd['right'].contour(slp_dict[i]['longitude'], slp_dict[i]['latitude'], slp_dict[i]['prmsl']/100,
                  levels=[1002],
                  colors=cols[i],transform=crs.PlateCarree())

cs1 = axd['right'].contour(slp_anl['longitude'], slp_anl['latitude'], slp_anl['prmsl']/100,
                  levels=np.arange(900,1060,6),
                  colors='k',transform=crs.PlateCarree())
axd['right'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(900,1060,6),fontsize=6,colors='k',zorder=-1)

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['right'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['right'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Add the gridlines
gls = axd['right'].gridlines(draw_labels=True, dms=True,
                   x_inline=False, y_inline=False,linestyle='dashed')
gls.top_labels=False
gls.bottom_labels=True
gls.right_labels=False
gls.left_labels=True
gls.xlocator = mticker.FixedLocator([-80, -75, -70, -65])
gls.ylocator = mticker.FixedLocator([30, 35, 40, 45])

axd['right'].set_title('1200 UTC 16 Nov 2018 ($t$ = 72 h)')
axd['right'].text(0.0, 1.0, '(b) SLP',transform=axd['right'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

plt.savefig('/p/work1/lloveras/nov2018/figs_pdf/figa1_gefs_errors.pdf',bbox_inches='tight')
plt.show()